# Demo

In [ ]:
import time
import ray
print(ray.__version__)

In [ ]:
ray.shutdown()
ray.init("ray://127.0.0.1:10001")

In [ ]:
from contextlib import contextmanager

@contextmanager
def timer():
    """Context manager to measure running time of code."""
    start = time.time()
    yield
    time_elapsed = round(time.time() - start)
    print(f"timer: took {time_elapsed} seconds")

In [ ]:
def get_instance_type():
    """Returns what instance type this function is running on."""
    import requests
    token = requests.put(
        "http://169.254.169.254/latest/api/token",
        headers={"X-aws-ec2-metadata-token-ttl-seconds": "21600"}
    ).text
    return requests.get(
        "http://169.254.169.254/latest/meta-data/instance-type",
        headers={"X-aws-ec2-metadata-token": token},
    ).text

---

In [ ]:
@ray.remote(
    num_cpus=1,
    memory=1024 ** 3,
)
def preprocess():
    time.sleep(1)
    return get_instance_type()

In [ ]:
with timer():
    print(ray.get(preprocess.remote()))

In [ ]:
from collections import Counter

with timer():
    print(Counter(
        ray.get([preprocess.remote() for _ in range(60)])
    ))

In [ ]:
with timer():
    print(Counter(
        ray.get([preprocess.remote() for _ in range(6000)])
    ))

---

In [ ]:
@ray.remote(num_gpus=1, accelerator_type="p2")
def train():
    time.sleep(1)
    return get_instance_type()

with timer():
    print(ray.get(train.remote()))

In [ ]:
@ray.remote(num_gpus=8, accelerator_type="p2")
def train_many():
    time.sleep(1)
    return get_instance_type()

with timer():
    print(ray.get(train_many.remote()))

In [ ]:
from ray.autoscaler.sdk import request_resources

# request_resources()

In [ ]:
cluster_resources = ray.cluster_resources()
CPUs = int(cluster_resources["CPU"])
memory = round(cluster_resources["memory"] / (1000 ** 3))
GPUs = round(cluster_resources.get("GPU", 0))
print(f"CPUs = {CPUs}")
print(f"memory = {memory}G")
print(f"GPUs = {GPUs}")

In [ ]:
!kubectl get no -L node.kubernetes.io/instance-type,karpenter.sh/capacity-type